# <span style="color:teal">RBFE Network - Analysis</span>


In [1]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np
import math
import pandas as pd
import networkx as nx
import yaml
from scipy.stats import sem as sem
from scipy.stats import bootstrap
from sklearn.metrics import mean_absolute_error as mae
import pickle
import tempfile
import itertools
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd 
from rdkit import Chem

# try w cinnabar
sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar
from cinnabar import wrangle,plotting
print(cinnabar.__file__)

try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest 

/home/anna/Documents/cinnabar/cinnabar/__init__.py
adding code to the pythonpath...


In [2]:
# folders
protein = "tyk2"
file_ext = "MBAR_alchemlyb_benchmark" # for results files

# define all the folder locations
bench_folder = f"/home/anna/Documents/benchmark"
main_folder = f"{bench_folder}/{protein}_benchmark"
out_folder = f"{main_folder}/outputs"
res_folder = f"../test/results"
temp_folder = f"{main_folder}/outputs/results/temp"
exp_folder = f"{bench_folder}/inputs/experimental"

# make folders that may not exist
folder_list = [res_folder, temp_folder]
for fold in folder_list:
    validate.folder_path(fold, create=True)

# files
net_file = f"{main_folder}/execution_model/network_lomap.dat"
exp_file = f"{exp_folder}/{protein}.yml"
exp_file_dat = f"{res_folder}/exp_data_{protein}.dat"

# OUTPUT
file_ext_out = "test" # for how files will be written

# files that will get written
comp_pert_file_name = f"computed_perturbations_average_{file_ext_out}"
cinnabar_file = f"cinnabar_format_{file_ext_out}"
# TODO add file names for saving graphs

In [3]:
res_dir = "/home/anna/Documents/benchmark/tyk2_benchmark/outputs"
res_obj = analysis_engines(res_dir, net_file=net_file, exp_file=exp_file, engines="SOMD")
res_obj.compute()
res_obj.exper_val_dict

engine input not recognised. Will use all engines.
no results output folder provided, writing all output to the 'results_directory'.


{'lig_ejm42': (-9.875042796569057, 0.18247842991081598),
 'lig_ejm49': (-7.823044291580318, 0.18452358795084045),
 'lig_ejm31': (-9.633320218997211, 0.1858893663539991),
 'lig_ejm48': (-9.087063209245514, 0.18452358795084045),
 'lig_ejm53': (-7.994549189742122, 0.18452358795084045),
 'lig_ejm47': (-9.788490911006289, 0.18275455833723164),
 'lig_jmc23': (-11.808150641219255, 0.18452358795084045),
 'lig_ejm46': (-11.419259655949855, 0.17907904179257983),
 'lig_ejm44': (-7.489423055117317, 0.17520781056282297),
 'lig_ejm45': (-9.645871445320283, 0.18313068534612764),
 'lig_jmc28': (-11.078583174659633, 0.18838478980330997),
 'lig_jmc30': (-11.037920060427668, 0.18236617963082224),
 'lig_ejm54': (-10.63128006862631, 0.18452358795084045),
 'lig_ejm55': (-9.292643737706989, 0.18452358795084134),
 'lig_ejm50': (-9.062726718781265, 0.18452358795084045),
 'lig_ejm43': (-8.340214497048866, 0.18296499294886548),
 'lig_jmc27': (-11.383117606839484, 0.18067733310175882)}

In [4]:
from cinnabar import wrangle,plotting
network = wrangle.FEMap(f"/home/anna/Documents/benchmark/tyk2_benchmark/outputs/cinnabar_SOMD.csv")

for node in network.graph.nodes(data=True):
    print(node[0], node[1])

0 {'name': 'lig_ejm45', 'exp_DG': -9.645871445320283, 'exp_dDG': 0.18313068534612764, 'calc_DG': 0.6628971518294626, 'calc_dDG': 0.09591119801726128}
1 {'name': 'lig_ejm53', 'exp_DG': -7.994549189742122, 'exp_dDG': 0.18452358795084045, 'calc_DG': 1.769333311164411, 'calc_dDG': 0.07158050409152511}
2 {'name': 'lig_ejm44', 'exp_DG': -7.489423055117317, 'exp_dDG': 0.17520781056282297, 'calc_DG': 1.5075937936610808, 'calc_dDG': 0.053013995917800524}
3 {'name': 'lig_ejm54', 'exp_DG': -10.63128006862631, 'exp_dDG': 0.18452358795084045, 'calc_DG': -1.1203935699722685, 'calc_dDG': 0.08913646490103168}
4 {'name': 'lig_ejm55', 'exp_DG': -9.292643737706989, 'exp_dDG': 0.18452358795084134, 'calc_DG': -0.15727890809445344, 'calc_dDG': 0.060928173333954554}
5 {'name': 'lig_ejm48', 'exp_DG': -9.087063209245514, 'exp_dDG': 0.18452358795084045, 'calc_DG': 0.8540954923784945, 'calc_dDG': 0.05972513580493125}
6 {'name': 'lig_ejm43', 'exp_DG': -8.340214497048866, 'exp_dDG': 0.18296499294886548, 'calc_DG':

In [4]:
# import networkanalysis
# import experiments
# import stats

exp_dicts = res_obj._get_exp_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/')
# for key in exp_dicts[0]:
#     print(f"{key} : {exp_lig_dict[key]}")

free_fwf_dict = res_obj._get_ana_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")

fwf_stats = res_obj._get_stats_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
print(fwf_stats)

Added additional data to 44 edges; added 0 new edges.
Added additional data to 43 edges; added 0 new edges.
R confidence is:   0.55 < 0.57 < 0.60
MUE confidence is: 0.83 < 0.86 < 0.88
Tau confidence is: 0.34 < 0.35 < 0.38
(array([0.57072062, 0.54659264, 0.59502589]), array([0.35294118, 0.33823529, 0.38235294]), array([0.85643673, 0.83117573, 0.88212284]))


In [ ]:
# cycle closures
for eng in engines:

    pert_dict = values_dict[eng]["pert_results"]
    cycle_closures = # get from the network graph object

    cycles = pipeline.analysis.make_dict_cycle_closures(pert_dict, cycle_closures)    

    print(f"{eng} cycle vals is {cycles[1]}")
    print(f"{eng} cycle mean is {cycles[2]}")
    print(f"{eng} cycle deviation is {cycles[3]}")

    values_dict[eng]["cycle_closures"] = (cycles[0]) # the cycles dict

In [ ]:
from cinnabar import wrangle,plotting
network = wrangle.FEMap(f"{res_folder}/{cinnabar_file}_{eng}.csv")

In [ ]:
for eng in engines:
    convert_cinnabar_file(values_dict[eng]["results_files"], values_dict["experimental"]["val_results"], f"{res_folder}/{cinnabar_file}_{eng}")

    network = wrangle.FEMap(f"{res_folder}/{cinnabar_file}_{eng}.csv")
    # plot the perturbations
    plotting.plot_DDGs(network.graph, filename=f"{res_folder}/DDGs_{file_ext_out}_{eng}.png", title=f"DDGs_{protein}_{file_ext_out}_{eng}")
    #plot the ligands
    plotting.plot_DGs(network.graph, filename=f"{res_folder}/DGs_{file_ext_out}_{eng}.png", title=f"DGs_{protein}_{file_ext_out}_{eng}")